In [6]:
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.fftpack as fftpack
from tqdm import tqdm
import os
import random 
# for reading and displaying images
#from skimage.io import imread
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score


# PyTorch libraries and modules
import torch
import warnings
from torch.autograd import Variable
import datatorch as dt
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms,utils
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim import *
import glob as glob

Tensor shape represents dimensions of a tensor, which is defined by the num of elements along each axis. Size is the total number of elements in the tensor (ex: 16983560). 
The shape error is saying the shape of the tensor I am trying to create by increasing the sample size from 10 to 20 
In flattened_size = 64 * 28 * 34 * 28, I am definingnthe shape of my tensor as 64 * 28 * 34 * 28 which equals 16983560
When I increase the sample size from 10 to 20, the input size of the tensor becomes larger than 16983560 resulting in the shape error 
to fix I need to update the flattened size var to match the new input size 

I can do this by calculating the flattened size dynamically based on the shape of the output from the second conv layer
Instead of hard coding flattened size, I can ensure the flattened size is always compatible with the input size 

Remove fully connected l;auer and add (play around with this) convolution layers opr binary interpolation layers 

look up what loss to use in this 




In [ ]:
#size check of mri 
img = nib.load('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/wneuromorphometrics_2068_mri.nii.gz')
dim = img.shape
print("dim of mri image:", dim) #(121, 145, 121)

#size check of segment 
img1 = nib.load('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_2068/021.nii.gz')
dim1 = img1.shape
print("dim of region image:", dim1) #(121, 145, 121)

img1 = nib.load('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_2068/022.nii.gz')


In [8]:
#NOT WORKING 
#dynamically calculate the flattened size works here but results in a RuntimeError: Given groups=1, weight of size [32, 1, 3, 3, 3], expected input[1, 4, 121, 145, 121] to have 1 channels, but got 4 channels instead when I run with first training code 
#and RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x1705984 and 10816x128) at line ---> 44 out = self.fc1(out)
num_classes = 2
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layer1 = self.conv_layer_set(1, 32)  # 3 input, 32 output
        self.conv_layer2 = self.conv_layer_set(32, 64)

        # calculate flattened size dynamically based on shape of output from conv_layer
        conv_output_shape = self.get_conv_output_shape((1, 1, 64, 64, 64))          #error: untimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [1, 1, 1, 64, 64, 64]
        self.flattened_size = conv_output_shape[1] * conv_output_shape[2] * conv_output_shape[3]
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

        # relu
        self.relu = nn.ReLU()
        # dropout
        self.drop = nn.Dropout(p=0.15)
        # batchnorm
        self.batchnorm = nn.BatchNorm1d(128)

    # define the actual structure of conv layer
    def conv_layer_set(self, in_c, out_c):
        # actual conv layer architecture
        conv_layer = nn.Sequential(
            nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),  # make padding smaller than 1/2 kernel size
            #I feel like the kernel size 3,3,3, is causing this error: weight of size [32, 1, 3, 3, 3], expected input[1, 4, 121, 145, 121] to have 1 channels, but got 4 channels instead
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=3, stride=2))  # nn.MaxPool3d(2,2,2))
        return conv_layer

    def get_conv_output_shape(self, shape):
        with torch.no_grad():
            x = torch.zeros(*shape) #changed this from (1,*shape) and now its working 
            x = self.conv_layer1(x)  #it's adding another 1 
            x = self.conv_layer2(x)
            return tuple(x.size())

    # forward pass through nn
    def forward(self, x):
        x = x.type('torch.FloatTensor').cuda()  # x is input to first conv layer
        out = self.conv_layer1(x)  # reassinging out is flowing from one layer to next in order

        out = self.conv_layer2(out) #mat1 and mat2 shapes cannot be multiplied (4x1705984 and 10816x128)

        out = out.view(out.size(0), -1)  # flattens output tensor to 1d vector with 2nd dim being -1, meaning its computed based on size of first dim and total num elements in tensor
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batchnorm(out)
        out = self.drop(out)
        out = self.fc2(out)
        return out  # return final output


n_iters = 4500 #manually chosen 

#creates cnn object by creating instance of CNNModel()
model = CNNModel().cuda() #error
# print("Model:",model)

#cross entropy loss 
error = nn.CrossEntropyLoss()

#SDG optimizer 
learning_rate = .001
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [7]:
#works when flattened size is hard coded 
num_classes = 2
# fileSize = 10   add something like this 
class CNNModel(nn.Module):
    def __init__(self, flattened_size): #maybe make flattened size a input argument to the class 
        super(CNNModel,self).__init__()
        self.conv_layer1 = self.conv_layer_set(1,32) #3 input, 32 output 
        self.conv_layer2 = self.conv_layer_set(32,64) 
        # self.flattened_size = flattened_size
        # self.flattened_size = None
        #32 in, 64 out 
        # calculate flattened size dynamically based on shape of output from conv_layer
        # conv_output_shape = self.get_conv_output.shape()
        # flattened_size = conv_output_shape[1]* conv_output_shape[2]*conv_output_shape[3]

        #hard coding the flattened size is not good idea bc anytime you change it changes
        flattened_size = 64 * 28 * 34 * 28 
        #it runs when I hardcode flattened size as this but not when I try to dynanmically calculate flattened size using self.flattenedsize 
        #equal to (4*6823936) took out 4 but its still saying [4,6823936] invalid for 6823936
        # self.fc1 = nn.Linear(4*512, 128) #this is too big for the cuda to handle  (4*512, 128) 6823936x128 (4*1705984, 128)
        #(512) is flatted num neurons from output of 2nd conv layer 
        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128,num_classes) 

        #relu 
        self.relu = nn.ReLU()
        #dropout
        self.drop = nn.Dropout(p=0.15)
        #batchnorm
        self.batchnorm = nn.BatchNorm1d(128)

#define the actual structure of conv layer 
    def conv_layer_set(self, in_c, out_c): #2 args input chan and output chan, when called above its 2,32
        #actual conv layer architecture 
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3,3,3), padding = 0), #make padding smaller than 1/2 kernel size 
        nn.ReLU(),
        nn.MaxPool3d(kernel_size = 3, stride =2)) #nn.MaxPool3d(2,2,2))
        return conv_layer
        
    #forward pass through nn 
    def forward(self,x):
        x = x.type('torch.FloatTensor').cuda() #x is input to first conv layer 
        # print("input tensor:",f'x {x.shape}')
        # print(x)
        out = self.conv_layer1(x) #reassinging out is flowing from one layer to next in order 
        # print("output of c1:",out.shape)
        out = self.conv_layer2(out)
        # print("output of c2:",out.shape) #input is output of l1
        out = out.view(out.size(0),-1) #flattens output tensor to 1d vector with 2nd dim being -1, meaning its computed based on size of first dim and total num elements in tensor 
        # out = out.view(out.size(0), 6823936) 
        # print("1D flattened tensor:",out.shape) 

        out = self.fc1(out) 
        # print("output of c1:",out)
        out = self.relu(out) 
        #relu 
        # print("output of relu:",out)
        out = self.batchnorm(out) 
        # print("output after batch norm:",out)
        #droput
        out = self.drop(out) 
        # print("output after dropout:",out)
        #final layer
        out = self.fc2(out) 
        # print("final output after f2:" ,out) 
        return out #return final output 


#defining hyperparameters 
n_iters = 4500 #manually chosen 

#creates cnn object by creating instance of CNNModel()
model = CNNModel(64 * 28 * 34 * 28).cuda() #this was the flattened size originally but I think it won't work if its changed 
# print("Model:",model)

#cross entropy loss 
error = nn.CrossEntropyLoss()

#SDG optimizer 
learning_rate = .001
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [ ]:
#modify data loader to search for images with naming convention masked_* 


In [15]:
#I think caitlin is iterating though all the folders and then deleting the ones she doesn't want, change so it matches certain ones 


#custom dataloader for 3d that reads csv file and load associated img file
class ImageDataSet(Dataset):
    def __init__(self,annotations_file,image_loader = None,transform=None):
        self.file = pd.read_csv(annotations_file) #only first row (one subj) 
        # print(self.file.shape)  #1 row, 19 col 
        #choose a random sample of 20 regions for training bc 142 is too large 
        #list to store nii files 
        self.image_files_total = []

        for subfolder in self.file["roi_path"]:
            #create list of nii files in current subfolder by joining path and file name 
            image_files = [os.path.join(subfolder,file) for file in os.listdir(subfolder)]

            #random_sample = random.sample(image_files,20)
            #not sure why this fixed the memory issue if the selg image files total doesn't have random sample in it 
            #print(len(random_sample))
            #adds image files to list 

            self.image_files_total = self.image_files_total + image_files
            # self.image_files_total = self.image_files_total + random_sample
        #print(type(self.image_files_total[:10]))
        
        #self.image_files_total = random.shuffle(self.image_files_total)
        self.image_files_total = self.image_files_total[:10] #if I change from 10 to 20, the tensor is now the wrong size  
        #shape '[4, 1, 121, 145, 121]' is invalid for input of size 16983560
        
        #if I increase this try decreasing batch size 

        # for entry in [entry for entry in self.image_files_total if not entry.startswith("masked")]:
        #to get the masked signal files change to the above line 
        for entry in [entry for entry in self.image_files_total if entry.endswith(".npy")]:
            self.image_files_total.remove(entry) #deletes .npy files in subfolder
            #deleted anything that doesn't start with masked 
        
        # print(self.image_files_total) 
        
        #defines var for transform arg above 
        self.transform = transform 

    def __len__(self):
        return len(self.image_files_total)
        # print(len(self.image_files_total))

    def __getitem__(self, index):
        image = nib.load((self.image_files_total[index])).get_fdata() #get_fdata retrives nii image as numpy array 
        image = torch.tensor(image, dtype=torch.float) #convert nparray to torch tensor

        #label (ID) is extracted by processing path of img file and finding the gender corresponding to it in the csv 

        person = self.image_files_total[index][0].split("/")[-1]  #[-2]
        person = person[:-4] #only taking the last 4 elements? 
        # print(label) #/to split path and get second to last element which is ID /data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_0331/001.npy 0331 is second last 

        #search for row with roi path matching label/id 
        loc = self.file[self.file["roi_path"].str.contains(person)]

        #extract gender value (first el) in this col 
        # print(loc["PTGENDER_demo"])
        gender = loc["PTGENDER_demo"].values[index] #fixed by replacing -1 with index 
        # print(gender)  #for this person I

        #convert gender from string to binary 
        #this is causing an error so I need to convert to tensor 
        
        # label = torch.nn.functional.one_hot(gender,2)
        ##takes long tensor with index value of shape(*) and returns tensor of shape (*,num_classes) that have zeros everywhere except where the index of the last dimension corresponding to value of input tensor which is 1 
        temp = []
        if gender == "F":
            label = 0
        else: 
            label = 1


        return {'label': label, 'gender': gender, 'image': image} #changed image to image_mag 


In [14]:
file = pd.read_csv("/home/mlauber/MandACollab/training_data.csv")
file.head()

,RID,PROGRESSION_CATEGORY,PROGRESSION_CATEGORY_2YR,PROGRESSES,APOE,TIME_TO_PROGRESSION,TIME_TO_FINAL_DX,TIMES,TIMES_ROUNDED,MMSCORE_mmse,abeta,ptau,tau,DX_VALUES,DX_DATES,STEPS_UNDER_1_YR,AGE,PTGENDER_demo,roi_path
0,5007,NaN,NaN,0.0,0.0,NaN,24.0,24.0,24.0,27.0,622.0,14.70,147.1,"MCI,MCI,MCI,MCI","0,6,12,24",True,72.0,M,/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/w...
1,4540,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,29.0,1896.0,18.67,194.8,MCI,0,True,71.0,F,/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/w...
2,2133,-1.0,0.0,1.0,2.0,84.0,96.0,84.0,84.0,30.0,413.9,28.71,328.3,"MCI,MCI,MCI,MCI,MCI,MCI,MCI,AD,AD","0,6,12,24,36,48,60,84,96",False,63.0,F,/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/w...
3,4674,NaN,0.0,0.0,1.0,NaN,84.0,84.0,84.0,24.0,843.1,31.89,329.1,"MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI","0,6,12,24,36,48,60,72,84",True,78.0,M,/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/w...
4,2073,NaN,0.0,0.0,0.0,NaN,72.0,72.0,72.0,29.0,1094.0,24.09,250.2,"MCI,MCI,MCI,MCI,MCI,NL,NL,NL","0,6,12,24,36,48,60,72",True,63.0,F,/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/w...


In [16]:
#call imagedataset function to load csv with prog and mri info 
TrainDataset = ImageDataSet("/home/mlauber/MandACollab/training_data.csv")
ValidationDataset =  ImageDataSet("/home/mlauber/MandACollab/validation_data.csv")
TestDataset = ImageDataSet("/home/mlauber/MandACollab/testing_data.csv")

In [14]:
# pd.read_csv('/home/mlauber/MandACollab/training_data.csv')

In [5]:
data_size = len(TrainDataset)

BATCH_size = 10 #increase in increments of 2, faster 

#shoukld be some multple of 2  #by specifying batch size of 20, its taking 20 random regions to train on? #cuda is out of memory with this size 
 
#use data loader to split into test and train 
Traindataloader = DataLoader(TrainDataset, batch_size=BATCH_size,shuffle=False, num_workers=0) #60% of data 
Testdataloader = DataLoader(TestDataset, batch_size=BATCH_size,shuffle=True, num_workers=0) #20%
Validationdataloader = DataLoader(ValidationDataset, batch_size=BATCH_size,shuffle=True, num_workers=0) #20%

Questions: 
batch_size = 20, is randomly selecting 20 regions to analyze?

Best way to finish training the cnn/how to tell when its ready to use 

If we're training it on gender, we can still use it to extract feature vectors from the subjects we used to train it? yes 

hard coding 0 to female 
do configurations file if you encounter the value do some mapping 
this makes code neat bc you dont need if else statement 
put everything into dict and map to correct valuye

In [13]:
# batch[('image')].max() 
# print(torch.max(batch)) 

In [11]:
#this is now not working 
#modified version: difference is  train = Variable(images) instead of original train = Variable(images.view(4,1,121,145,121)) 
count = 0
loss_list = [] 
iteration_list = []
accuracy_list = []
num_epochs = 30   #cuda out of batch size with this many epochs 
iteration_info = []

for epoch in range(num_epochs):
    #training 
    for i, sample_batched in enumerate(Traindataloader):
        labels = sample_batched['label'].cuda()
        images = sample_batched['image'].cuda()
        print(images.shape) #torch.Size([4, 121, 145, 121])
        
        #original code 

        # train = Variable(images.view(8,121,145,121)) 

        # train = Variable(images.view(4,1,121,145,121)) 
        train = Variable(images)
        labels = Variable(labels) 

        #reset gradients to 0 before starting backprop
        optimizer.zero_grad()
        #forward pass on current batch of training images 
        outputs = model(train)                   
        #diff btw prediction and ground truth  
        loss = error(outputs,labels)
        #backprop 
        loss.backward()
        #update param
        optimizer.step() 
        count += 1
    #store loss and iteration number
        loss_list.append(loss.data)
        iteration_list.append(count)
    
    # Calculate accuracy on the current batch of training data
    #calc accuracy per epoch 
    correct = 0
    total = 0
    with torch.no_grad(): #temporarily sets the gradient computation off 
    # get the predicted classes from the model
        _, predicted = torch.max(outputs.data, 1)
    # compare the predicted classes to the ground truth labels
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = correct / total
        accuracy_list.append(accuracy)
        iteration_info.append("Iteration:{} Loss:{} Accuracy:{} %".format(count,loss.data,accuracy))

for info in iteration_info:
    print(info)


#ideally loss should decrease and accuracy should increase 
#might need to increase the sample size from 10 to more  

NameError: name 'Traindataloader' is not defined

Output of fcl2: tensor([[-2.0090,  1.3628],
        [ 0.6485, -0.8229],
        [ 1.7357, -0.9457],
        [-0.9922,  0.3083]]

Are the activations from the fcl, which maps high level features extracted by the convolutional layers to the final output class scores 

Each row in the tensor are the activations for a specific input from the batch. With this one there are 4 rows (from 4 inputs) and 2 activations each which correspond to the strenght of activation for each of the 2 output classes, which is the predicted probability of the input being in that class. 
IE -2.0090 for class 1, 1.3628 for class 2, so it's prediciting its class 2 

The class with higher score is the predicted label, negative values mean lower probability for a class and positive is higher prob. 
These values depend on the learned weights in the FC layer and the features generated from previous layers.

Next step is convert these activations into class predictions 

to do: write a separate validation function after the model runs to track if it is improving or not 

Find a way to increase the batch size and have the shape still fit 

After I run it separately, eventually integrate within the model loop

so that it tracks and prints every iteration 

write validagtion function and integrate somewhere in the code

train func, then val, then call within 

see manually validation score after each iteration 

if still improving run epochs tp 10 

if validation not improving, its good

try increasing number samples 

In [25]:
!CUDA_VISIBLE_DEVICES=1

In [19]:
#original full training code
#CNN model training 
#this version actually works with the hard coded flattened size code 

count = 0
loss_list = [] 
iteration_list = []
accuracy_list = []
num_epochs = 30

for epoch in range(num_epochs):
    #training 
    for i, sample_batched in enumerate(Traindataloader):
        labels = sample_batched['label'].cuda()
        images = sample_batched['image'].cuda()
        # print(images.shape)
        #original code 
        # train = Variable(images.view(8,121,145,121)) 
        #take out the 1 
        train = Variable(images.view(4,1,121,145,121)) #.view reshapes tensor 
        labels = Variable(labels) 

        #reset gradients to 0 before starting backprop
        optimizer.zero_grad()
        #forward pass on current batch of training images 
        outputs = model(train)                   
        #diff btw prediction and ground truth  
        loss = error(outputs,labels)
        #backprop 
        loss.backward()
        #update param
        optimizer.step() 
        count += 1
    #store loss and iteration number
        loss_list.append(loss.data)
        iteration_list.append(count)
    
    # Calculate accuracy on the current batch of training data
        correct = 0
        total = 0
        # with torch.no_grad(): #temporarily sets the gradient computation off 
        # # get the predicted classes from the model
        #     _, predicted = torch.max(outputs.data, 1)
        # # compare the predicted classes to the ground truth labels
        #     total += labels.size(0)
        #     correct += (predicted == labels).sum().item()
        #     accuracy = correct / total
        #     accuracy_list.append(accuracy)
        #     print("Iteration:{} \nLoss:{} \nAccuracy:{} %".format(count,loss.data,accuracy))
        #     #print out preditiction 
        with torch.no_grad(): #temporarily sets the gradient computation off 
    # get the predicted classes from the model
            _, predicted = torch.max(outputs.data, 1)
    # compare the predicted classes to the ground truth labels
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = correct / total
            accuracy_list.append(accuracy)
            iteration_info.append("Iteration:{} Loss:{} Accuracy:{} %".format(count,loss.data,accuracy))

for info in iteration_info:
    print(info)


FT applied, image shape: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.

matmul error is being caused by 
try reshaping it to (batch size, num_features) 

Consistent 1% accuracy might be because model is biased towards one class and is only predicting the one class
If it was random guessing then you would expect 50% accuracy 